In [1]:
# https://www.tensorflow.org/tutorials/audio/simple_audio

In [1]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models

from IPython import display


2022-12-18 21:15:49.903374: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-18 21:15:50.407447: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64:/usr/local/TensorRT-8.5.1.7/lib:
2022-12-18 21:15:50.407606: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64:/usr/local/TensorRT-8.5.1.7/lib:
2022-12-18

In [2]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [6]:
DATASET_PATH = '/BiO/github/big_files/audio/data/mini_speech_commands'

data_dir = pathlib.Path(DATASET_PATH)
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='/BiO/github/big_files/audio', cache_subdir='data')

182082353/182082353 [==============================] - 3s 0us/step


In [7]:
data_dir

PosixPath('/BiO/github/big_files/audio/data/mini_speech_commands')

In [13]:
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[commands != "README.md"]
print("Ccommandsds:" , commands)

Ccommandsds: ['down' 'go' 'left' 'no' 'right' 'stop' 'up' 'yes']


In [14]:
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(directory=data_dir,
                                                              batch_size=64,
                                                              validation_split=0.2,
                                                              seed = 0, 
                                                              output_sequence_length=16000,
                                                              subset = "both")

label_names = np.array(train_ds.class_names)
print()
print("label names" , label_names)

Found 8000 files belonging to 8 classes.
Using 6400 files for training.
Using 1600 files for validation.

label names ['down' 'go' 'left' 'no' 'right' 'stop' 'up' 'yes']


2022-12-18 21:27:17.292921: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-18 21:27:17.373751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-18 21:27:17.373894: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-18 21:27:17.375232: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [17]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 16000, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [18]:
train_ds.element_spec

(TensorSpec(shape=(None, 16000, None), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [21]:
def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels

train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

ValueError: in user code:

    File "/tmp/ipykernel_19676/1967832569.py", line 2, in squeeze  *
        audio = tf.squeeze(audio, axis=-1)

    ValueError: Can not squeeze dim[1], expected a dimension of 1, got 16000 for '{{node Squeeze}} = Squeeze[T=DT_FLOAT, squeeze_dims=[-1]](args_0)' with input shapes: [?,16000].
